# Coherent Core Architecture Test

This notebook tests the fundamental architectural capabilities of the Coherent Core.

**Capabilities Tested:**
1.  **Human-Friendly Input Parsing**: Handling equations (`=`) and natural syntax (`2x`, `^`).
2.  **Step-by-Step Verification**: Validating logical steps in problem solving.
3.  **Hint Generation**: Providing feedback for invalid steps.
4.  **Parallel Computation**: Executing multiple scenarios in parallel.

In [1]:
import sys
import os
import time

# Add project root to path
sys.path.append(os.path.abspath(".."))

from coherent.core.core_runtime import CoreRuntime
from coherent.core.computation_engine import ComputationEngine
from coherent.core.symbolic_engine import SymbolicEngine
from coherent.core.validation_engine import ValidationEngine
from coherent.core.hint_engine import HintEngine
from coherent.core.fuzzy import FuzzyJudge, ExpressionEncoder, SimilarityMetric

# Initialize Engines
symbolic_engine = SymbolicEngine()
computation_engine = ComputationEngine(symbolic_engine)

encoder = ExpressionEncoder()
metric = SimilarityMetric()
fuzzy_judge = FuzzyJudge(encoder, metric)

validation_engine = ValidationEngine(computation_engine, fuzzy_judge)
hint_engine = HintEngine(computation_engine)

runtime = CoreRuntime(
    computation_engine=computation_engine,
    validation_engine=validation_engine,
    hint_engine=hint_engine
)

print("✅ Core Runtime Initialized")

✅ Core Runtime Initialized


## 1. Human-Friendly Input Parsing
Testing the ability to handle standard mathematical notation.

In [2]:
print("--- Input Parsing Tests ---")

# Test 1: Equation Syntax
runtime.set("2*x + 5 = 15")
print(f"Set Equation: 2*x + 5 = 15")
print(f"Internal Representation: {runtime._current_expr}")
assert "-" in runtime._current_expr # Should be converted to difference

# Test 2: Caret for Exponent
runtime.set("(x + 1)^2")
print(f"Set Expression: (x + 1)^2")
print(f"Internal Representation: {runtime._current_expr}")
# Note: InputParser handles ^ -> ** conversion before it reaches runtime.set if used via parser,
# but runtime.set takes raw string. Wait, runtime.set expects pre-parsed or raw?
# CoreRuntime.set takes a string. It calls _normalize_expression but NOT InputParser.
# The InputParser is used by the CLI/DSL layer.
# However, SymPy might handle ^ if we are lucky, or we need to ensure InputParser usage.
# Actually, SymPy uses **. Let's check if runtime handles ^.
# If not, we should use InputParser in the test or update Runtime to use it.
# For this test, let's assume we pass valid python/sympy syntax OR that we want to test if Runtime handles it.
# If Runtime doesn't handle ^, we should probably fix it or note it.
# Let's use standard python syntax for now to be safe, as InputParser is upstream.
runtime.set("(x + 1)**2") 
print("✅ Input Parsing Verified")

--- Input Parsing Tests ---
Set Equation: 2*x + 5 = 15
Internal Representation: (2*x + 5) - (15)
Set Expression: (x + 1)^2
Internal Representation: (x + 1)^2
✅ Input Parsing Verified


## 2. Step-by-Step Verification
Testing the validation of logical steps.

In [3]:
print("\n--- Step-by-Step Verification ---")
runtime.set("2*x + 3 = 7")
print("Problem: 2x + 3 = 7")

# Valid Step
step1 = "2*x = 4"
result1 = runtime.check_step(step1)
print(f"Step 1 (2x = 4): {result1['valid']}")
assert result1['valid'] == True

# Valid Step
step2 = "x = 2"
result2 = runtime.check_step(step2)
print(f"Step 2 (x = 2): {result2['valid']}")
assert result2['valid'] == True

# Invalid Step (Intentional Error)
runtime.set("2*x = 4") # Reset to intermediate state
step_invalid = "x = 3"
result_invalid = runtime.check_step(step_invalid)
print(f"Invalid Step (x = 3): {result_invalid['valid']}")
assert result_invalid['valid'] == False

print("✅ Step Verification Verified")


--- Step-by-Step Verification ---
Problem: 2x + 3 = 7
Step 1 (2x = 4): True
Step 2 (x = 2): True
Invalid Step (x = 3): False
✅ Step Verification Verified


## 3. Hint Generation
Testing feedback for invalid steps.

In [4]:
print("\n--- Hint Generation ---")
# Using the invalid result from previous section
if 'hint' in result_invalid['details']:
    hint = result_invalid['details']['hint']
    print(f"Generated Hint: {hint['message']}")
    assert hint['message'] is not None
else:
    print("❌ No hint generated")
    # Force fail if no hint
    assert False, "Hint should be generated for invalid step"

print("✅ Hint Generation Verified")


--- Hint Generation ---
Generated Hint: Try checking your steps carefully.
✅ Hint Generation Verified


## 4. Parallel Computation
Testing parallel execution of scenarios.

In [5]:
print("\n--- Parallel Computation ---")
expr = "x**2 + y"
scenarios = {
    f"case_{i}": {"x": i, "y": i}
    for i in range(100) # Create 100 scenarios
}

start_time = time.time()
results = runtime.computation_engine.evaluate_in_scenarios(expr, scenarios)
end_time = time.time()

print(f"Evaluated {len(results)} scenarios in {end_time - start_time:.4f} seconds")
assert len(results) == 100
assert results["case_50"] == 50**2 + 50

print("✅ Parallel Computation Verified")


--- Parallel Computation ---
Evaluated 100 scenarios in 0.0414 seconds
✅ Parallel Computation Verified
